In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import gensim.downloader

In [3]:
word2vec_rus = gensim.downloader.load('word2vec-ruscorpora-300')

In [4]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [5]:
#import fasttext.util

In [6]:
#fasttext.util.download_model('ru', if_exists='ignore')

In [7]:
#!curl https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

In [8]:
from navec import Navec
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [12]:
########################################33
def get_word_embeddings(sentence):
    tokens = nltk.word_tokenize(sentence)
    unk = navec['<unk>']
    embeddings = [navec.get(token.lower(), unk) for token in tokens]
    return np.mean(embeddings, axis=0)

In [13]:
# Upload DataFrame:
df = pd.read_csv('intents.csv', delimiter=';', encoding='cp1251')

In [14]:
df.loc[:, 'encoddings'] = df.loc[:, 'content'].apply(get_word_embeddings)

In [15]:
df[:10]

,content,intent,encoddings
0,Спасибо нет,Благодарность,"[-0.105404586, -0.06919877, -0.026303, 0.04673..."
1,спасибо за ответ. мне этого достаточно пока чт...,Благодарность,"[-0.066874005, -0.12562214, -0.08311224, -0.05..."
2,"спасибо, нет",Благодарность,"[0.001167655, 0.07729654, 0.028063437, -0.0310..."
3,Спасибо!,Благодарность,"[0.13164875, 0.17279321, 0.124180056, -0.03025..."
4,Благодарю,Благодарность,"[0.1128936, 0.21874672, 0.11713439, 0.03774131..."
5,Благодарю! И Вам.,Благодарность,"[0.16147722, 0.14516404, 0.03254261, 0.0006748..."
6,Спасибо.,Благодарность,"[0.13164875, 0.17279321, 0.124180056, -0.03025..."
7,Спасибо! Мне удобней получить информацию здесь...,Благодарность,"[-0.0324358, 0.00021839142, 0.01969277, 0.0097..."
8,спасибо,Благодарность,"[0.048985362, -0.02470075, 0.1115638, 0.126034..."
9,"спасибо, вам тоже",Благодарность,"[0.1305559, 0.086980596, 0.040952656, 0.012191..."


In [16]:
res = df.groupby('intent')['encoddings'].agg(list).reset_index()

In [17]:
res

,intent,encoddings
0,Благодарность,"[[-0.105404586, -0.06919877, -0.026303, 0.0467..."
1,Возраст,"[[-0.14902718, -0.12232632, -0.20918293, -0.08..."
2,Вопрос по курсу,"[[-0.06172894, 0.018506097, -0.11030652, -0.02..."
3,Вышка,"[[0.3071782, 0.24292763, -0.07325962, -0.06845..."
4,Документы,"[[0.00082808733, -0.048323154, -0.065854326, 0..."
5,Записаться,"[[-0.010293789, -0.28740612, -0.10405151, 0.10..."
6,Интенсив,"[[-0.0035616904, -0.17219493, -0.12022232, 0.0..."
7,Консультация,"[[0.07698171, -0.012375734, 0.038861837, -0.02..."
8,Не определен,"[[0.16772683, -0.39048105, -0.3405094, 0.02039..."
9,Не перезвонил МОП,"[[0.15987946, -0.114601076, 0.017500028, -0.06..."


In [18]:
res['center']=res['encoddings'].apply(lambda x: np.mean(x, axis=0))

In [19]:
res

,intent,encoddings,center
0,Благодарность,"[[-0.105404586, -0.06919877, -0.026303, 0.0467...","[0.055651687, 0.016190393, 0.010570393, -0.000..."
1,Возраст,"[[-0.14902718, -0.12232632, -0.20918293, -0.08...","[0.033503354, -0.08212651, -0.09825751, -0.076..."
2,Вопрос по курсу,"[[-0.06172894, 0.018506097, -0.11030652, -0.02...","[0.04660241, -0.10331272, -0.06325631, -0.0501..."
3,Вышка,"[[0.3071782, 0.24292763, -0.07325962, -0.06845...","[-0.03492254, -0.08585773, -0.023116931, -0.04..."
4,Документы,"[[0.00082808733, -0.048323154, -0.065854326, 0...","[0.0036946777, 0.016767332, 0.020225242, -0.10..."
5,Записаться,"[[-0.010293789, -0.28740612, -0.10405151, 0.10...","[0.040979147, -0.10112871, -0.022057118, -0.04..."
6,Интенсив,"[[-0.0035616904, -0.17219493, -0.12022232, 0.0...","[0.05643052, -0.031518497, 0.0027437946, -0.06..."
7,Консультация,"[[0.07698171, -0.012375734, 0.038861837, -0.02...","[0.015950345, -0.09594794, -0.05830809, -0.022..."
8,Не определен,"[[0.16772683, -0.39048105, -0.3405094, 0.02039...","[0.009878048, -0.14926215, -0.06331843, -0.016..."
9,Не перезвонил МОП,"[[0.15987946, -0.114601076, 0.017500028, -0.06...","[0.0069739176, -0.115699105, -0.044321157, 0.0..."


In [20]:
import spacy
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Calculate distances between classes
class_distances = {}
unique_classes = res["intent"].unique()
for i in range(len(unique_classes)):
    for j in range(i, len(unique_classes)):
        class1 = unique_classes[i]
        class2 = unique_classes[j]
        print(res[res['intent']==class1].iloc[0, 2])
        dist = cosine(res[res['intent']==class1].iloc[0,2], res[res['intent']==class2].iloc[0,2])
        class_distances[(class1, class2)] = dist

[ 5.56516871e-02  1.61903929e-02  1.05703929e-02 -6.09736948e-04
 -5.52565567e-02 -9.41201076e-02  5.62762544e-02  9.53958035e-02
  3.30641754e-02 -1.26790270e-01 -7.79416636e-02  1.83589924e-02
 -1.41747743e-01  1.58025436e-02 -4.51440364e-02  4.18052189e-02
  8.79542611e-04 -8.61166716e-02  2.92574912e-02 -7.83963650e-02
 -1.32920489e-01  6.73472062e-02  1.83046417e-04  1.57065958e-01
  3.59458961e-02  9.76906791e-02 -3.96097936e-02  8.82093012e-02
 -4.12857942e-02 -6.64728973e-03 -1.57326877e-01 -5.16701676e-02
  2.99150288e-01  2.08117500e-01  1.49367794e-01  4.87040691e-02
 -1.30311906e-01 -3.44689526e-02  1.06958874e-01  2.66123675e-02
  3.26679554e-03 -1.49311841e-01  1.37585029e-01 -8.59185383e-02
 -5.97222857e-02  1.30264589e-03  9.13302675e-02 -1.82150695e-02
 -8.59409496e-02 -1.94057077e-01 -1.77196637e-01  8.20709914e-02
  4.18449044e-02  1.19807579e-01  3.50631215e-03 -1.09043665e-01
  1.04772476e-02  4.14194800e-02  1.23485871e-01  9.56442431e-02
  7.83006381e-03 -1.58568

In [21]:
# Print distances between classes
first =[]
second =[]
rast = []
for (class1, class2), dist in class_distances.items():
    first.append(class1)
    second.append(class2)
    rast.append(dist)
    print(f"Distance between {class1} and {class2}: {dist:.4f}")

Distance between Благодарность and Благодарность: 0.0000
Distance between Благодарность and Возраст: 0.2849
Distance between Благодарность and Вопрос по курсу: 0.2814
Distance between Благодарность and Вышка: 0.3892
Distance between Благодарность and Документы: 0.3458
Distance between Благодарность and Записаться: 0.2792
Distance between Благодарность and Интенсив: 0.2529
Distance between Благодарность and Консультация: 0.1849
Distance between Благодарность and Не определен: 0.2055
Distance between Благодарность and Не перезвонил МОП: 0.1918
Distance between Благодарность and Обучение: 0.3729
Distance between Благодарность and Ответ на недозвон: 0.1611
Distance between Благодарность and Отзывы: 0.2434
Distance between Благодарность and Отказ: 0.1166
Distance between Благодарность and Отписаться: 0.3361
Distance between Благодарность and Подарки: 0.2552
Distance between Благодарность and Прощание: 0.0647
Distance between Благодарность and Работать у вас/нанять: 0.2646
Distance between Б

In [22]:
pd.DataFrame({'first': first, 'second': second, 'rast': rast})

,first,second,rast
0,Благодарность,Благодарность,0.000000
1,Благодарность,Возраст,0.284945
2,Благодарность,Вопрос по курсу,0.281377
3,Благодарность,Вышка,0.389152
4,Благодарность,Документы,0.345802
...,...,...,...
523,Формат обучения,Цена,0.082428
524,Формат обучения,Юрик,0.123863
525,Цена,Цена,0.000000
526,Цена,Юрик,0.091093
